In [3]:
#111
from transformers import AutoModel, AutoTokenizer

HUGGINGFACE_MODEL_PATH = "klue/bert-base" # bert-base-multilingual-cased
model = AutoModel.from_pretrained(HUGGINGFACE_MODEL_PATH)
tokenizer = AutoTokenizer.from_pretrained(HUGGINGFACE_MODEL_PATH)

/home/soma1/miniconda3/envs/tf/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at klue/bert-base were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSe

In [ ]:
#22222
#load trainset
#beginning of the real training!
import json
import pandas as pd
with open('./processed_human/merged.json', mode='rt', encoding='utf-8-sig') as f:
    train_dataset = json.load(f)

train_dataset_list = [{'text':data['text'], 'label':data['label']} for data in train_dataset]
train_df = pd.DataFrame(train_dataset_list)
train_df.head()

,text,label
0,머리말,12
1,일러두기,12
2,목차,12
3,학년 학기,12
4,즐거운 마음으로,12


In [6]:
#222222222222222233333
from sklearn.preprocessing import LabelEncoder
import numpy as np

label_encoder = LabelEncoder()
label_encoder.fit(train_df['label'])
num_labels = len(label_encoder.classes_)

train_df['encoded_label'] = np.asarray(label_encoder.transform(train_df['label']), dtype=np.int32)
train_df.head()

,text,label,encoded_label
0,머리말,12,0
1,일러두기,12,0
2,목차,12,0
3,학년 학기,12,0
4,즐거운 마음으로,12,0


In [26]:
train_df.groupby(by=['encoded_label']).count()

,text,label
encoded_label,,
0,2500,2500
1,3527,3527
2,2907,2907


In [7]:
#4444
#remove labels from the text
#set 20% of the set as a validation set
train_texts = train_df["text"].to_list() # Features (not-tokenized yet)
train_labels = train_df["encoded_label"].to_list() # Labels

from sklearn.model_selection import train_test_split
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=0.2, random_state=0)

In [8]:
#5555
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(HUGGINGFACE_MODEL_PATH)

# Tokenizing
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

In [9]:
#66666
import tensorflow as tf

# trainset-set
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
))

# validation-set
val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_labels
))

2023-07-12 06:36:46.375159: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-12 06:36:46.931838: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-07-12 06:36:47.480539: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-12 06:36:47.481188: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA n

In [11]:
#7777
from transformers import TFBertForSequenceClassification

num_labels = len(label_encoder.classes_)
model = TFBertForSequenceClassification.from_pretrained(HUGGINGFACE_MODEL_PATH, num_labels=num_labels, from_pt=True)

optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model.compile(optimizer=optimizer, loss=model.hf_compute_loss, metrics=['accuracy'])

2023-07-12 06:37:10.876312: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.b

In [13]:
#88888
from tensorflow.keras.callbacks import EarlyStopping

callback_earlystop = EarlyStopping(
    monitor="val_accuracy", 
    min_delta=0.001, # the threshold that triggers the termination (acc should at least improve 0.001)
    patience=2)

model.fit(
    train_dataset.shuffle(1000).batch(16), epochs=5, batch_size=16,
    validation_data=val_dataset.shuffle(1000).batch(16),
    callbacks = [callback_earlystop]
)

Epoch 1/5


2023-07-12 06:39:30.837534: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int32 and shape [7147,29]
	 [[{{node Placeholder/_1}}]]
2023-07-12 06:39:30.837736: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype int32 and shape [7147,29]
	 [[{{node Placeholder/_2}}]]
2023-07-12 06:39:30.877414: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:417] Loaded runtime CuDNN library: 8.5.0 but source was compiled with: 8.6.0.  CuDNN library needs to have matching major version and equal or higher minor version. If using a binary install, upgrade your CuDNN library.  

InternalError: Graph execution error:

Detected at node 'StatefulPartitionedCall_199' defined at (most recent call last):
    File "/home/soma1/miniconda3/envs/tf/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/soma1/miniconda3/envs/tf/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/home/soma1/miniconda3/envs/tf/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/soma1/miniconda3/envs/tf/lib/python3.9/site-packages/traitlets/config/application.py", line 1043, in launch_instance
      app.start()
    File "/home/soma1/miniconda3/envs/tf/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 712, in start
      self.io_loop.start()
    File "/home/soma1/miniconda3/envs/tf/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "/home/som

In [18]:
#if 7777 and 8888 doens't work
from transformers import TFTrainer, TFTrainingArguments

training_args = TFTrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=5,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    eval_steps=500
)

with training_args.strategy.scope():
    trainer_model = TFBertForSequenceClassification.from_pretrained(HUGGINGFACE_MODEL_PATH, num_labels=num_labels, from_pt=True)

trainer = TFTrainer(
    model=trainer_model,                 # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset             # evaluation dataset

)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
#999999
trainer.train()

2023-07-12 06:58:16.552271: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32 and shape [7147,29]
	 [[{{node Placeholder/_0}}]]
2023-07-12 06:58:16.552519: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype int32 and shape [7147,29]
	 [[{{node Placeholder/_2}}]]
2023-07-12 06:58:16.552879: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:786] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorSliceDataset/_4"
op: "TensorSliceDatase

In [28]:
#10 save model
import os
MODEL_NAME = 'fine-tuned-klue-bert-base'
MODEL_SAVE_PATH = os.path.join("_model", MODEL_NAME) # change this to your preferred location

if os.path.exists(MODEL_SAVE_PATH):
    print(f"{MODEL_SAVE_PATH} -- Folder already exists \n")
else:
    os.makedirs(MODEL_SAVE_PATH, exist_ok=True)
    print(f"{MODEL_SAVE_PATH} -- Folder create complete \n")

# save tokenizer, model
model.save_pretrained(MODEL_SAVE_PATH)
tokenizer.save_pretrained(MODEL_SAVE_PATH)

_model/fine-tuned-klue-bert-base -- Folder create complete 



('_model/fine-tuned-klue-bert-base/tokenizer_config.json',
 '_model/fine-tuned-klue-bert-base/special_tokens_map.json',
 '_model/fine-tuned-klue-bert-base/vocab.txt',
 '_model/fine-tuned-klue-bert-base/added_tokens.json')

In [55]:
#11 load pretrained
from transformers import TextClassificationPipeline

# Load Fine-tuning model
loaded_tokenizer = BertTokenizer.from_pretrained(MODEL_SAVE_PATH)
loaded_model = TFBertForSequenceClassification.from_pretrained(MODEL_SAVE_PATH)

text_classifier = TextClassificationPipeline(
    tokenizer=loaded_tokenizer, 
    model=loaded_model, 
    framework='tf',
    return_all_scores=True
)

with open('_data/eval.json', mode='rt', encoding='utf-8-sig') as f:
    test_dataset = json.load(f)

#test_dataset_list = [{'text':clean_text(data['title']), 'label':data['label']} for data in test_dataset]
test_df = pd.DataFrame(test_dataset)
test_df.head()

Some layers from the model checkpoint at _model/fine-tuned-klue-bert-base were not used when initializing TFBertForSequenceClassification: ['dropout_37']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at _model/fine-tuned-klue-bert-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.
/home/soma1/miniconda3/envs/tf/lib/python3.9/

,text,label
0,한 때 평범한 아이였던 서아는 이제 3살이 되었습니다. 그녀는 항상 웃는 얼굴로 주...,LABEL_0
1,"어느 날, 서아는 엄마와 함께 병원에 갔어요. 그곳에서 의사가 된다는 이야기를 들었...",LABEL_0
2,서아는 의사가 되기 위해 열심히 공부하기 시작했어요. 그녀는 의사에 대해 많이 알고...,LABEL_0
3,"한 여름날, 서아는 학교에 다니기 시작했어요. 그곳에서는 많은 친구들과 함께 놀고 ...",LABEL_0
4,"서아는 학교에서도 의사가 되기 위해 열심히 노력했어요. 그녀는 친구들을 도와주고, ...",LABEL_0


In [59]:
#12 print out evaluated values
predicted_label_list_1 = []
predicted_score_list_1 = []
predicted_label_list_0 = []
predicted_score_list_0 = []
predicted_label_list_2 = []
predicted_score_list_2 = []
predicted_label_list = []
predicted_score_list = []

for text in test_df['text']:
    # predict
    preds_list = text_classifier(text)[0]

    sorted_preds_list = sorted(preds_list, key=lambda x: x['score'], reverse=True)
    print(sorted_preds_list)

    predicted_label_list.append(sorted_preds_list[0]["label"]) # label
    predicted_score_list.append(sorted_preds_list[0]["score"]) # score
    if  sorted_preds_list[0]["label"] == "LABEL_1" :
        predicted_label_list_1.append(sorted_preds_list[0]["label"]) # label
        predicted_score_list_1.append(sorted_preds_list[0]["score"]) # score4
    elif sorted_preds_list[0]["label"] == "LABEL_0" :
        predicted_label_list_0.append(sorted_preds_list[0]["label"]) # label
        predicted_score_list_0.append(sorted_preds_list[0]["score"]) # score4
    else:
        predicted_label_list_2.append(sorted_preds_list[0]["label"]) # label
        predicted_score_list_2.append(sorted_preds_list[0]["score"]) # score4
    
    if  sorted_preds_list[1]["label"] == "LABEL_1" :
        predicted_label_list_1.append(sorted_preds_list[1]["label"]) # label
        predicted_score_list_1.append(sorted_preds_list[1]["score"]) # score4
    elif sorted_preds_list[1]["label"] == "LABEL_0" :
        predicted_label_list_0.append(sorted_preds_list[1]["label"]) # label
        predicted_score_list_0.append(sorted_preds_list[1]["score"]) # score4
    else:
        predicted_label_list_2.append(sorted_preds_list[1]["label"]) # label
        predicted_score_list_2.append(sorted_preds_list[1]["score"]) # score4
    
    if  sorted_preds_list[2]["label"] == "LABEL_1" :
        predicted_label_list_1.append(sorted_preds_list[2]["label"]) # label
        predicted_score_list_1.append(sorted_preds_list[2]["score"]) # score4
    elif sorted_preds_list[2]["label"] == "LABEL_0" :
        predicted_label_list_0.append(sorted_preds_list[2]["label"]) # label
        predicted_score_list_0.append(sorted_preds_list[2]["score"]) # score4
    else:
        predicted_label_list_2.append(sorted_preds_list[2]["label"]) # label
        predicted_score_list_2.append(sorted_preds_list[2]["score"]) # score4
        

test_df['label_pred'] = predicted_label_list
test_df['score_score'] = predicted_score_list
test_df['label_0'] = predicted_label_list_0
test_df['score_0'] = predicted_score_list_0
test_df['label_1'] = predicted_label_list_1
test_df['score_1'] = predicted_score_list_1
test_df['label_2'] = predicted_label_list_2
test_df['score_2'] = predicted_score_list_2

test_df.head()

[{'label': 'LABEL_1', 'score': 0.4937843978404999}, {'label': 'LABEL_0', 'score': 0.3096281588077545}, {'label': 'LABEL_2', 'score': 0.19658741354942322}]
[{'label': 'LABEL_1', 'score': 0.4864237308502197}, {'label': 'LABEL_2', 'score': 0.3020857274532318}, {'label': 'LABEL_0', 'score': 0.21149055659770966}]
[{'label': 'LABEL_1', 'score': 0.5384460687637329}, {'label': 'LABEL_2', 'score': 0.25357601046562195}, {'label': 'LABEL_0', 'score': 0.20797793567180634}]
[{'label': 'LABEL_1', 'score': 0.5052276849746704}, {'label': 'LABEL_0', 'score': 0.2941308319568634}, {'label': 'LABEL_2', 'score': 0.20064149796962738}]
[{'label': 'LABEL_1', 'score': 0.47602197527885437}, {'label': 'LABEL_2', 'score': 0.2821674346923828}, {'label': 'LABEL_0', 'score': 0.24181054532527924}]
[{'label': 'LABEL_1', 'score': 0.5490851402282715}, {'label': 'LABEL_0', 'score': 0.24093133211135864}, {'label': 'LABEL_2', 'score': 0.20998351275920868}]
[{'label': 'LABEL_1', 'score': 0.5416573882102966}, {'label': 'LABE

,text,label,label_0,score_0,label_1,score_1,label_2,score_2,label_pred,score_score
0,한 때 평범한 아이였던 서아는 이제 3살이 되었습니다. 그녀는 항상 웃는 얼굴로 주...,LABEL_0,LABEL_0,0.309628,LABEL_1,0.493784,LABEL_2,0.196587,LABEL_1,0.493784
1,"어느 날, 서아는 엄마와 함께 병원에 갔어요. 그곳에서 의사가 된다는 이야기를 들었...",LABEL_0,LABEL_0,0.211491,LABEL_1,0.486424,LABEL_2,0.302086,LABEL_1,0.486424
2,서아는 의사가 되기 위해 열심히 공부하기 시작했어요. 그녀는 의사에 대해 많이 알고...,LABEL_0,LABEL_0,0.207978,LABEL_1,0.538446,LABEL_2,0.253576,LABEL_1,0.538446
3,"한 여름날, 서아는 학교에 다니기 시작했어요. 그곳에서는 많은 친구들과 함께 놀고 ...",LABEL_0,LABEL_0,0.294131,LABEL_1,0.505228,LABEL_2,0.200641,LABEL_1,0.505228
4,"서아는 학교에서도 의사가 되기 위해 열심히 노력했어요. 그녀는 친구들을 도와주고, ...",LABEL_0,LABEL_0,0.241811,LABEL_1,0.476022,LABEL_2,0.282167,LABEL_1,0.476022


In [60]:
#13 save as json
file_path = '_data/eval_out.json'
test_df.to_json(file_path, orient='records', force_ascii=False, lines = True)

print(f"DataFrame saved to {file_path}")

DataFrame saved to _data/eval_out.json


In [ ]:
#Below are some trials and errors. not so much important
#Trial and error includes: checking tensorflow version, preprocessing the input data, etc.

In [ ]:
import kss

text = '딥 러닝 자연어 처리가 재미있기는 합니다. 그런데 문제는 영어보다 한국어로 할 때 너무 어렵습니다. 이제 해보면 알걸요?'

print('한국어 문장 토큰화 :',kss.split_sentences(text))

In [ ]:
#kss tokenizing test
inp = './processed/in.txt'
outp = './processed/out.txt'
with open(inp, 'r', encoding='utf-8') as file:
    content = file.read()
print('한국어 문장 토큰화 :',kss.split_sentences(content))
s = kss.split_sentences(content)

with open(outp, 'w', encoding='utf-8') as file:
    file.writelines(s)


In [ ]:
#12
import pandas as pd
import numpy as np
inp = './processed/12_out.txt'
outp = './processed/12_out.txt'
with open(inp, 'r', encoding='utf-8') as file:
    content = file.read()

#with open('_data/ynat-v1.1_train.json', mode='rt', encoding='utf-8-sig') as f:
#    train_dataset = json.load(f)

#train_dataset_list = [{'text':data['title'], 'label':data['label']} for data in train_dataset]
train_df = pd.DataFrame(content)
train_df.head()

In [ ]:
#make file into json
import json

def txt_to_json(input, output, label):
    with open(input, 'r', encoding='utf-8') as file:
        lines = file.read().splitlines()
    data = []
    for s in lines:
        data.append({'text': s, 'label': label})
    with open(output, 'w', encoding='utf-8') as file:
        json.dump(data, file, indent = 4)


no = '56'
input = './processed_human/' + no + '_out_b.txt'
output = './processed_human/' + no + '_out.json'
label = no
txt_to_json(input, output, label)

In [ ]:
#2 
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(train_df['label'])
num_labels = len(label_encoder.classes_)

train_df['encoded_label'] = np.asarray(label_encoder.transform(train_df['label']), dtype=np.int32)
train_df.head()

In [ ]:
#remove blank in front of lines

def txt_to_json(input, output):
    with open(input, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    lines = [line.lstrip() for line in lines]

    with open(output, 'w', encoding='utf-8') as file:
        file.writelines(lines)

no = '56'
input = './processed_human/' + no + '_out.txt'
output = './processed_human/' + no + '_out_b.txt'
txt_to_json(input, output)

In [ ]:

from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(train_df['label'])
num_labels = len(label_encoder.classes_)

In [ ]:
train_labels

In [ ]:
from transformers import TFTrainer, TFTrainingArguments, TFBertForSequenceClassification

training_args = TFTrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=5,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs'            # directory for storing logs
)

with training_args.strategy.scope():
    trainer_model = TFBertForSequenceClassification.from_pretrained(HUGGINGFACE_MODEL_PATH, num_labels=num_labels, from_pt=True)

trainer = TFTrainer(
    model=trainer_model,                 # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset             # evaluation dataset
)

In [10]:
#check tf
import tensorflow as tf

from tensorflow.python.client import device_lib
def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]
print(get_available_devices())

tf.sysconfig.get_build_info() 

['/device:CPU:0', '/device:GPU:0']


2023-07-12 06:37:06.559214: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-12 06:37:06.559361: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-12 06:37:06.559485: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

OrderedDict([('cpu_compiler', '/dt9/usr/bin/gcc'),
             ('cuda_compute_capabilities',
              ['sm_35', 'sm_50', 'sm_60', 'sm_70', 'sm_75', 'compute_80']),
             ('cuda_version', '11.8'),
             ('cudnn_version', '8'),
             ('is_cuda_build', True),
             ('is_rocm_build', False),
             ('is_tensorrt_build', True)])

In [ ]:
tf.test.gpu_device_name()

In [ ]:
import tensorflow as tf

from tensorflow.python.client import device_lib
def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]
print(get_available_devices())

tf.sysconfig.get_build_info() 

In [ ]:
from tensorflow.python.platform import build_info as tf_build_info
print(tf_build_info.cuda_version_number)
# 9.0 in v1.10.0
print(tf_build_info.cudnn_version_number)
# 7 in v1.10.0


In [45]:
predicted_label_list = []
predicted_score_list = []

for text in test_df['text']:
    # predict
    preds_list = text_classifier(text)[0]
    sorted_preds_list = sorted(preds_list, key=lambda x: x['score'], reverse=True)

    print(sorted_preds_list)
    predicted_label_list.append(sorted_preds_list[0]["label"]) # label
    predicted_score_list.append(sorted_preds_list[1]) # score

test_df['pred'] = predicted_label_list
test_df['score'] = predicted_score_list
test_df.head()

[{'label': 'LABEL_1', 'score': 0.4937843978404999}, {'label': 'LABEL_0', 'score': 0.3096281588077545}, {'label': 'LABEL_2', 'score': 0.19658741354942322}]
[{'label': 'LABEL_1', 'score': 0.4864237308502197}, {'label': 'LABEL_2', 'score': 0.3020857274532318}, {'label': 'LABEL_0', 'score': 0.21149055659770966}]
[{'label': 'LABEL_1', 'score': 0.5384460687637329}, {'label': 'LABEL_2', 'score': 0.25357601046562195}, {'label': 'LABEL_0', 'score': 0.20797793567180634}]
[{'label': 'LABEL_1', 'score': 0.5052276849746704}, {'label': 'LABEL_0', 'score': 0.2941308319568634}, {'label': 'LABEL_2', 'score': 0.20064149796962738}]
[{'label': 'LABEL_1', 'score': 0.47602197527885437}, {'label': 'LABEL_2', 'score': 0.2821674346923828}, {'label': 'LABEL_0', 'score': 0.24181054532527924}]
[{'label': 'LABEL_1', 'score': 0.5490851402282715}, {'label': 'LABEL_0', 'score': 0.24093133211135864}, {'label': 'LABEL_2', 'score': 0.20998351275920868}]
[{'label': 'LABEL_1', 'score': 0.5416573882102966}, {'label': 'LABE

,text,label,pred,score
0,한 때 평범한 아이였던 서아는 이제 3살이 되었습니다. 그녀는 항상 웃는 얼굴로 주...,LABEL_0,LABEL_1,"{'label': 'LABEL_0', 'score': 0.3096281588077545}"
1,"어느 날, 서아는 엄마와 함께 병원에 갔어요. 그곳에서 의사가 된다는 이야기를 들었...",LABEL_0,LABEL_1,"{'label': 'LABEL_2', 'score': 0.3020857274532318}"
2,서아는 의사가 되기 위해 열심히 공부하기 시작했어요. 그녀는 의사에 대해 많이 알고...,LABEL_0,LABEL_1,"{'label': 'LABEL_2', 'score': 0.25357601046562..."
3,"한 여름날, 서아는 학교에 다니기 시작했어요. 그곳에서는 많은 친구들과 함께 놀고 ...",LABEL_0,LABEL_1,"{'label': 'LABEL_0', 'score': 0.2941308319568634}"
4,"서아는 학교에서도 의사가 되기 위해 열심히 노력했어요. 그녀는 친구들을 도와주고, ...",LABEL_0,LABEL_1,"{'label': 'LABEL_2', 'score': 0.2821674346923828}"
